# **hkl_soleil** K4CV

The *kappa* ($\kappa$) diffractometer {ref}`geometry
<geometries-hkl_soleil-k4cv>` replaces the $\chi$-ring on an Eulerian 4-circle
diffractometer with a $\kappa$ stage which holds the *phi* stage. The $\kappa$
stage is tilted at angle $\alpha$ (typically 50 degrees) from the $\omega$
stage.


![K4CV geometry](../_static/k4cv.png)

This is the *hkl_soleil* *K4CV* [geometry](../diffractometers.rst):

axis    | moves    | rotation axis    | vector
---     | ---      | ---              | ---
komega  | sample   | {math}`-\vec{y}` | `[0 -1 0]`
kappa   | sample   | {math}`\vec{x}`  | `[0 -0.6428 -0.7660]`
kphi    | sample   | {math}`-\vec{y}` | `[0 -1 0]`
tth     | detector | {math}`-\vec{y}` | `[0 -1 0]`

* xrays incident on the {math}`\vec{x}` direction (1, 0, 0)

See: *hkl_soleil* [documentation](https://people.debian.org/~picca/hkl/hkl.html) for more details.

## Define _this_ diffractometer

Use the **hklpy2** `creator()` function to create a diffractometer
object.  The diffractometer object will have simulated rotational axes.

We'll provide the geometry and solver names.
By convention, the `name` keyword is the same as the object name.

See the [geometry tables](../diffractometers.rst) for
a more complete description of the available diffractometers.

Create the Python diffractometer object (`k4cv`).  We choose this name to avoid any confusion with the diffractometer's `kappa` axis.

In [1]:
import hklpy2

k4cv = hklpy2.creator(name="k4cv", geometry="K4CV", solver="hkl_soleil")

## Add a sample with a crystal structure

In [2]:
from hklpy2.user import add_sample, calc_UB, cahkl, cahkl_table, pa, set_diffractometer, setor, wh

set_diffractometer(k4cv)
add_sample("silicon", a=hklpy2.SI_LATTICE_PARAMETER)

Sample(name='silicon', lattice=Lattice(a=5.431, system='cubic'))

## Setup the UB orientation matrix using *hklpy*

Define the crystal's orientation on the diffractometer using 
the 2-reflection method described by [Busing & Levy, Acta Cryst 22 (1967) 457](https://www.psi.ch/sites/default/files/import/sinq/zebra/PracticalsEN/1967-Busing-Levy-3-4-circle-Acta22.pdf).

### Diffractometer wavelength

Set the diffractometer's X-ray wavelength.  This will be used for both reflections.  `k4cv.wavelength` is an
ophyd [Signal](https://blueskyproject.io/ophyd/user/reference/signals.html). Use
its `.put()` method.

In [3]:
k4cv.beam.wavelength.put(1.54)

### Specify the first reflection

Provide the set of angles that correspond with the reflection's Miller indices: (_hkl_)

The `setor()` (set orienting reflection) method uses the diffractometer's wavelength *at the time it is called*.  (To add reflections at different wavelengths, add a `wavelength=1.0` keyword argument with the correct value.)  

In [4]:
r1 = setor(4, 0, 0, tth=-69.0966, komega=55.4507, kappa=0, kphi=-90)

### Specify the second reflection

In [5]:
r2 = setor(0, 4, 0, tth=-69.0966, komega=-1.5950, kappa=134.7658, kphi=123.3554)

### Compute the *UB* orientation matrix

The `calc_UB()` method returns the computed **UB** matrix.

In [6]:
calc_UB(r1, r2)

[[2.0191835e-05, -8.3752177e-05, -1.156906934347],
 [0.0, -1.156906934523, 8.3752177e-05],
 [-1.156906937379, -1.462e-09, -2.0191835e-05]]

## Report our setup

In [7]:
pa()

diffractometer='k4cv'
HklSolver(name='hkl_soleil', version='5.1.2', geometry='K4CV', engine_name='hkl', mode='bissector')
Sample(name='silicon', lattice=Lattice(a=5.431, system='cubic'))
Reflection(name='r_bf45', h=4, k=0, l=0)
Reflection(name='r_b967', h=0, k=4, l=0)
Orienting reflections: ['r_bf45', 'r_b967']
U=[[1.7453293e-05, -7.2393184e-05, -0.999999997227], [0.0, -0.99999999738, 7.2393184e-05], [-0.999999999848, -1.263e-09, -1.7453292e-05]]
UB=[[2.0191835e-05, -8.3752177e-05, -1.156906934347], [0.0, -1.156906934523, 8.3752177e-05], [-1.156906937379, -1.462e-09, -2.0191835e-05]]
constraint: -180.0 <= komega <= 180.0
constraint: -180.0 <= kappa <= 180.0
constraint: -180.0 <= kphi <= 180.0
constraint: -180.0 <= tth <= 180.0
Mode: bissector
beam={'class': 'WavelengthXray', 'source_type': 'Synchrotron X-ray Source', 'energy': 8.050921976530415, 'wavelength': 1.54, 'energy_units': 'keV', 'wavelength_units': 'angstrom'}
h=0, k=0, l=0
komega=0, kappa=0, kphi=0, tth=0


## Check the orientation matrix

Perform checks with `forward()` ($hkl$ to angle) and
`inverse()` (angle to $hkl$) computations to verify the diffractometer
will move to the same positions where the reflections were identified.

### Constrain one of the motors

* keep `kphi` in the negative range
* allow for slight roundoff errors

First, apply [constraints](../concepts/constraints.rst) to the `kphi`
rotational motor. Constraints are part of the diffractometer's `core`-level
support.

---

**Note**: A constraint does not limit the range of the motor, it *constrains*
the choice of solutions from the `forward()` computation.

---

In [8]:
k4cv.core.constraints["kphi"].limits = (-180, 0.001)
k4cv.core.constraints

['-180.0 <= komega <= 180.0', '-180.0 <= kappa <= 180.0', '-180.0 <= kphi <= 0.001', '-180.0 <= tth <= 180.0']

### (400) reflection test

1. Check the `inverse()` (angles -> (_hkl_)) computation.
1. Check the `forward()` ((_hkl_) -> angles) computation.

#### Check `inverse()` at (400)

To calculate the (_hkl_) corresponding to a given set of motor angles,
call `k4cv.inverse()`.

The _hkl_ values are provided as a Python [namedtuple](https://docs.python.org/3/library/collections.html#collections.namedtuple) structure.

In [9]:
print(f"{k4cv.real_axis_names=}")
# Specify values, by correct order of names.
k4cv.inverse(55.4507, 0, -90, -69.0966)

k4cv.real_axis_names=['komega', 'kappa', 'kphi', 'tth']


Hklpy2DiffractometerPseudoPos(h=6.159871816956, k=0, l=0)

#### Check `forward(400)`

Compute the angles necessary to position the diffractometer
for the given reflection.

---

**Note**:

For the `forward()` computation, more than one set of angles may be used to reach the same crystal reflection.  This test will report the *default* selection.  The *default* selection (which may be changed through methods described in module :mod:`hklpy2.ops`) is the first solution.

function | returns
--- | ---
`cahkl()` | The *default* solution
`cahkl_table()` | Table of all allowed solutions.

---

Before calling `forward()`, make sure we are using the desired operations
*mode*.  `"bissector"` maintains $\omega = (2\theta) / 2$

In [10]:
k4cv.core.mode = "bissector"

Here we print the *default* solution (the one returned by calling `cahkl()`.

In [11]:
cahkl(4, 0, 0)

Hklpy2DiffractometerRealPos(komega=55.450879097241, kappa=-1.0856e-07, kphi=-90.000999946633, tth=-69.098241875299)

---

**Note**: `cahkl()` is a shortcut to `k4cv.forward()`.

In [12]:
k4cv.forward(4, 0, 0)

Hklpy2DiffractometerRealPos(komega=55.450879097241, kappa=-1.0856e-07, kphi=-90.000999946632, tth=-69.0982418753)

---

Show the table of *all* `forward()` solutions for $(4\ 0\ 0)$ and $(0\ 4\ 0)$ allowed by the current constraints.  Since this function accepts a *list* of $hkl$  reflections, extra Python syntax is applied.

In [13]:
cahkl_table((4, 0, 0), (0, 4, 0))

======= = ======== ===== ======= ========
(hkl)   # komega   kappa kphi    tth     
======= = ======== ===== ======= ========
(4 0 0) 1 55.4509  0.0   -90.001 -69.0982
(4 0 0) 2 -55.4509 0.0   -90.001 69.0982 
======= = ======== ===== ======= ========



### (040) reflection test

Repeat the `inverse` and `forward` calculations for the
second orientation reflection.

#### Check `inverse()` at (040)

In [14]:
k4cv.inverse(-1.5950, 134.7568, 123.3554, -69.0966)

Hklpy2DiffractometerPseudoPos(h=-6.8203037e-05, k=3.999916773745, l=-0.000264650749)

#### Check `forward(040)`

In [15]:
k4cv.forward(0, 4, 0)

Hklpy2DiffractometerRealPos(komega=-1.587888157865, kappa=134.745974867529, kphi=-57.03958619065, tth=-69.098241325651)

## Scan in reciprocal space using Bluesky

To scan with Bluesky, we need more setup.

In [16]:
from bluesky import RunEngine
from bluesky import SupplementalData
from bluesky.callbacks.best_effort import BestEffortCallback
import bluesky.plans as bp
import databroker

bec = BestEffortCallback()
bec.disable_plots()
cat = databroker.temp().v2
sd = SupplementalData()

RE = RunEngine({})
RE.md = {}
RE.preprocessors.append(sd)
RE.subscribe(cat.v1.insert)
RE.subscribe(bec)

1

Setup the `RE` to save the `k4cv` {class}`~hklpy2.blocks.configure.Configuration` with every run.

In [17]:
crw = hklpy2.ConfigurationRunWrapper(k4cv)
RE.preprocessors.append(crw.wrapper)

### (_h00_) scan near (400)

In this example, we have no detector.  Still, we add the diffractometer
object in the detector list so that the _hkl_ and motor positions will appear
as columns in the table.

In [18]:

k4cv.move(4, 0, 0)
wh()
RE(bp.scan([k4cv], k4cv.h, 3.9, 4.1, 5))

wavelength=1.54
h=4.0, k=0, l=0
komega=55.4509, kappa=0, kphi=-90.001, tth=-69.0982


Transient Scan ID: 1     Time: 2025-04-09 22:36:10
Persistent Unique Scan ID: 'b189d75f-6353-49ed-94b5-6f66f6af3d80'
New stream: 'primary'
+-----------+------------+------------+----------------------+------------------+------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |     k4cv_h | k4cv_beam_wavelength | k4cv_beam_energy |     k4cv_k |     k4cv_l | k4cv_komega | k4cv_kappa |  k4cv_kphi |   k4cv_tth |
+-----------+------------+------------+----------------------+------------------+------------+------------+-------------+------------+------------+------------+
|         1 | 22:36:10.7 |      3.900 |                1.540 |            8.051 |     -0.000 |     -0.000 |      56.431 |     -0.000 |    -90.001 |    -67.137 |
|         2 | 22:36:10.7 |      3.950 |                1.540 |            8.051 |     -0.000 |     -0.000 |      55.943 |     -0.00

('b189d75f-6353-49ed-94b5-6f66f6af3d80',)

### (_hk0_) scan near (440)

Scan between the two orientation reflections. Need to keep $\varphi\ge0$ to avoid
big jumps during the scan.

In [19]:
k4cv.core.constraints["kphi"].limits = -0.001, 180

k4cv.move(4, 4, 0)
RE(bp.rel_scan([k4cv], k4cv.h, -0.2, 0.2, k4cv.k, 0.2, -0.2, 10))



Transient Scan ID: 2     Time: 2025-04-09 22:36:10
Persistent Unique Scan ID: 'f9fa8f8e-a5dc-408c-bc8d-179990452219'
New stream: 'primary'
+-----------+------------+------------+------------+----------------------+------------------+------------+-------------+------------+------------+------------+
|   seq_num |       time |     k4cv_h |     k4cv_k | k4cv_beam_wavelength | k4cv_beam_energy |     k4cv_l | k4cv_komega | k4cv_kappa |  k4cv_kphi |   k4cv_tth |
+-----------+------------+------------+------------+----------------------+------------------+------------+-------------+------------+------------+------------+
|         1 | 22:36:10.9 |      3.800 |      4.200 |                1.540 |            8.051 |      0.000 |    -121.557 |    -63.947 |    111.866 |   -106.839 |
|         2 | 22:36:10.9 |      3.844 |      4.156 |                1.540 |            8.051 |      0.000 |    -121.861 |    -63.054 |    111.524 |   -106.763 |
|         3 | 22:36:11.0 |      3.889 |      4.111 |  

('f9fa8f8e-a5dc-408c-bc8d-179990452219',)